In [242]:
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
import cleantext
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from skimage.io import imread
from skimage.transform import resize
import nltk
from nltk.util import ngrams
import torch.nn.functional as F
import torch.nn as nn
import ast
from torch.nn.utils.rnn import pad_sequence
import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as t
import torchvision.models as mm
import skimage
import torchvision.datasets as datasets
from torch.utils.data import Dataset,DataLoader
import torchtext
from torchtext.data.utils import get_tokenizer

## Task 1: Sentence level Sentiment Analysis

In [243]:
class TextPreprocess(Dataset):
    def __init__(self,df):
        self.df=df
        self.text=self.preprocess(df['raw'])
    def __len__(self):
        return len(self.text)
    def __getitem__(self,ind):
        text=self.text[ind]
        return text
    def preprocess(self,text):
        tokenize=get_tokenizer('basic_english')
        tt=[]
        for i in text:
            tt.append(cleantext.clean(str(i),clean_all=True,stemming=True,stopwords=True,lowercase=True,numbers=True,punct=True))
        vectorizer=TfidfVectorizer(tokenizer=tokenize)
        embeddings=vectorizer.fit_transform(tt)
        emb=[]
        for i in embeddings:
            emb.append(torch.from_numpy(i.toarray()[0]))
        padded=pad_sequence(emb,batch_first=True)
        return padded
imagesname=[]
path=r'C:\Users\HP\Downloads\sentiment\sentiment.csv'
for i in os.listdir(r"C:/Users/HP/Downloads/sentiment/sentiment_images"):
    imagesname.append(i)
df=pd.read_csv(path)
df=df[df['filename'].isin(imagesname)]
df=df.reset_index()
dataset=TextPreprocess(df)
dataloader=DataLoader(dataset,batch_size=32,shuffle=False)

In [244]:
# Assigning Processed Text to a Dataframe
textfeatures = []
with torch.no_grad():
    for i in dataloader:
        for j in i:
            textfeatures.append(j.tolist())
df['Processed Text']=textfeatures
textfeatures=torch.tensor(textfeatures)
sent_train=df['sentiment']
sent_test=df['sentiment']

In [245]:
#Image Preprocessing and Feature Extraction using Vgg
features=[]
path=r"C:/Users/HP/Downloads/sentiment/"
preprocess=t.Compose([t.Resize((224,224)),t.ToTensor(),t.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])])
dataset=datasets.ImageFolder(path,preprocess)
dataloader=torch.utils.data.DataLoader(dataset,batch_size=32,shuffle=False)
vgg=mm.vgg16(pretrained=True)
with torch.no_grad():
    for i,_ in dataloader:
        feature=vgg(i)
        features.append(feature)
features=torch.cat(features,dim=0)

C:\Users\HP\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HP\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [246]:
#Mapping Features to its respective FileNames in the df
ii=[]
for i in os.listdir(r"C:/Users/HP/Downloads/sentiment/sentiment_images"):
    ii.append(i)
f=[]
for i in df['filename']:
    for j in range(0,len(ii)):
        if i==ii[j]:
            f.append(features[j])
df['features']=f

In [247]:
#Test_Train Split
df1=df[(df['split']=='train')]
df1=df1.reset_index()
df2=df[(df['split']=='val')]
df2=df2.reset_index()
df3=df[(df['split']=='test')]
df3=df3.reset_index()
image_train=df1['features']
text_train=df1['Processed Text']
sent_train=df1['sentiment']
imagevalfeatures=df2['features']
textval=df3['Processed Text']
sent_val=df2['sentiment']
imagetestfeatures=df3['features']
texttest=df3['Processed Text']
sent_test=df3['sentiment']
image_train=list(image_train)
imagefeatures=torch.stack(image_train)
text_train=list(text_train)
textfeatures=torch.tensor(text_train)
textval=list(textval)
textval=torch.tensor(textval)
imagevalfeatures=list(imagevalfeatures)
imagevalfeatures=torch.stack(imagevalfeatures)
imagetestfeatures=list(imagetestfeatures)
imagetestfeatures=torch.stack(imagetestfeatures)
texttest=list(texttest)
texttest=torch.tensor(texttest)

In [248]:
#Creating a Dataloader for both image and text features
class Text_Image:
    def __init__(self,image,text,sent):
        self.image=image
        self.text=text
        self.sent=sent
    def __len__(self):
        return len(self.image)
    def __getitem__(self,index):
        image=self.image[index]
        text=self.text[index]
        sent=self.sent[index]
        return image,text,sent
dataset=Text_Image(imagefeatures,textfeatures,sent_train)
CombineDataloader=DataLoader(dataset,batch_size=43,shuffle=False)
dataset=Text_Image(imagevalfeatures,textval,sent_val)
valDataloader=DataLoader(dataset,batch_size=43,shuffle=False)
class Test:
    def __init__(self,image,text,sent):
        self.image=image
        self.text=text
        self.sent=sent
    def __len__(self):
        return len(self.image)
    def __getitem__(self,index):
        image=self.image[index]
        text=self.text[index]
        sent=self.sent[index]
        return image,text,sent
testdataset=Test(imagetestfeatures,texttest,sent_test)
testCombineDataloader=DataLoader(testdataset,batch_size=43,shuffle=True)

In [249]:
#MultiModal Architecture
class imagemodel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten=nn.Flatten()
        self.linear1=nn.Linear(1000,800)
        self.act_fn=nn.Sigmoid()
        self.linear2=nn.Linear(800,640)
        self.linear3=nn.Linear(640,320)
        self.linear4=nn.Linear(320,256)
    def forward(self,x):
        x=self.flatten(x)
        x=self.linear1(x)
        x=self.act_fn(x)
        x=self.linear2(x)
        x=self.act_fn(x)
        x=self.linear3(x)
        x=self.act_fn(x)
        x=self.linear4(x)
        x=self.act_fn(x)
        return x
class textmodel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten=nn.Flatten()
        self.linear1=nn.Linear(3324,800)
        self.act_fn=nn.Sigmoid()
        self.linear2=nn.Linear(800,640)
        self.linear3=nn.Linear(640,320)
        self.linear4=nn.Linear(320,256)
    def forward(self,x):
        x=self.flatten(x)
        x=self.linear1(x)
        x=self.act_fn(x)
        x=self.linear2(x)
        x=self.act_fn(x)
        x=self.linear3(x)
        x=self.act_fn(x)
        x=self.linear4(x)
        x=self.act_fn(x)
        return x
class fuse(nn.Module):
    def __init__(self,textsmodel,imagesmodel,inputsize):
        super().__init__()
        self.flatten=nn.Flatten()
        self.imagemodel=imagesmodel
        self.textmodel=textsmodel
        self.linear1=nn.Linear(inputsize,256)
        self.linear2=nn.Linear(256,128)
        self.linear3=nn.Linear(128,80)
        self.linear4=nn.Linear(80,64)
        self.linear5=nn.Linear(64,32)
        self.linear6=nn.Linear(32,16)
        self.linear7=nn.Linear(16,8)
        self.linear8=nn.Linear(8,2)
        self.act_fn=nn.Sigmoid() 
    def forward(self,images,texts):
        texts=self.textmodel(texts.float())
        images=self.imagemodel(images.float())
        o=torch.cat((texts,images),dim=1)
        self.flatten=nn.Flatten()
        output=self.linear1(o)
        output=self.act_fn(output)
        output=self.linear2(output)
        output=self.act_fn(output)
        output=self.linear3(output)
        output=self.act_fn(output)
        output=self.linear4(output)
        output=self.act_fn(output)
        output=self.linear5(output)
        output=self.act_fn(output)
        output=self.linear6(output)
        output=self.act_fn(output)
        output=self.linear7(output)
        output=self.act_fn(output)
        output=self.linear8(output)
        output=self.act_fn(output)
        return output
imagesmodel=imagemodel()
textsmodel=textmodel()

In [253]:
#Training
model=fuse(textsmodel,imagesmodel,256+256)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)
vc=nn.CrossEntropyLoss()
validationoptimizer=torch.optim.Adam(model.parameters(),lr=0.01)
i=0
for epoch in range(10):
    model.train()
    trainloss=0.0
    i=i+1
    for images,texts,labels in CombineDataloader:
        outputs=model(images,texts)
        loss=criterion(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        trainloss=trainloss+loss.item()
    model.eval()
    vloss=0.0
    with torch.no_grad():
        for images,texts,labels in valDataloader:
            validationoptimizer.zero_grad()
            outputs = model(images, texts)
            loss=vc(outputs,labels)
            vloss=vloss+loss.item()
    print('Epoch : ',i)
    print('Training Loss :',trainloss)
    print('Validation Loss: ',vloss)

Epoch :  1
Training Loss : 309.81534814834595
Validation Loss:  62.863512337207794
Epoch :  2
Training Loss : 309.8139919042587
Validation Loss:  62.86428481340408
Epoch :  3
Training Loss : 309.81359881162643
Validation Loss:  62.86484855413437
Epoch :  4
Training Loss : 309.8135393857956
Validation Loss:  62.865243792533875
Epoch :  5
Training Loss : 309.8135803937912
Validation Loss:  62.865494310855865
Epoch :  6
Training Loss : 309.81364530324936
Validation Loss:  62.86566388607025
Epoch :  7
Training Loss : 309.8137018084526
Validation Loss:  62.86576807498932
Epoch :  8
Training Loss : 309.81374484300613
Validation Loss:  62.86584252119064
Epoch :  9
Training Loss : 309.813774228096
Validation Loss:  62.865888476371765
Epoch :  10
Training Loss : 309.8137977719307
Validation Loss:  62.865917563438416


In [254]:
#Saving the Model
torch.save(model.state_dict(),"trained_model.pth")
#Evaluate
model=fuse(textsmodel,imagesmodel,256+256)
model.load_state_dict(torch.load("trained_model.pth"))
model.eval()
pred=[]
label=[]
with torch.no_grad():
    for images,texts,labels in testCombineDataloader:
        outputs=model(images,texts)
        for i,j in zip(torch.argmax(outputs,axis=1),labels):
            pred.append(i)
            label.append(j)
accuracy=metrics.accuracy_score(label,pred)
precision=metrics.precision_score(label,pred)
recall=metrics.recall_score(label,pred)
f1score=metrics.f1_score(label,pred)
print('Accuracy :',accuracy*100)
print('Recall :',recall)
print('Precision :',precision)
print('F1 Score :',f1score)

Accuracy : 54.807692307692314
Recall : 1.0
Precision : 0.5480769230769231
F1 Score : 0.7080745341614907


In [255]:
#Testing
torch.save(model.state_dict(),"trained_model.pth")
model=fuse(textsmodel,imagesmodel,256+256)
criteria=nn.CrossEntropyLoss()
model.load_state_dict(torch.load("trained_model.pth"))
model.eval()
loss=0.0
with torch.no_grad():
    for images,texts,labels in testCombineDataloader:
        outputs=model(images,texts)
        losss=criteria(outputs,labels)
        loss=loss+losss.item()
print("Test Loss: ",loss)

Test Loss:  255.50272405147552


## Task 2: Word level Sentiment Analysis

In [230]:
#Text Bigrams
class textbigrams:
    def __init__(self,df):
        self.df=df
        self.text=self.preprocess(df['raw'])
    def __len__(self):
        return len(self.text)
    def __getitem__(self,ind):
        text=self.text[ind]
        return text
    def preprocess(self,text):
        tt=[]
        for i in text:
            tt.append(cleantext.clean(str(i),clean_all=True,stemming=True,stopwords=True,lowercase=True,numbers=True,punct=True))
        bigram=[]
        for i in tt:
            j=word_tokenize(i)
            bigrams=list(ngrams(j,2))
            bigram.append(bigrams)
        index={}
        for i in bigram:
            for j in i:
                if j not in index:
                    index[j]=len(index)
        embedded=[]
        for i in bigram:
            emb=torch.tensor([index[j] for j in i])
            embedded.append(emb)
        padded=pad_sequence(embedded,batch_first=True,padding_value=0)
        return padded
imagesname=[]
df = pd.read_csv(r'C:\Users\HP\Downloads\sentiment\sentiment.csv')
for i in os.listdir(r"C:/Users/HP/Downloads/sentiment/sentiment_images"):
    imagesname.append(i)
df=df[df['filename'].isin(imagesname)]
df=df.reset_index()
df1=df[(df['split']=='train')]
df1=df1.reset_index()
dataset=textbigrams(df1)
dataloader=DataLoader(dataset,batch_size=32,shuffle=False)
bigramfeatures=[]
with torch.no_grad():
    for i in dataloader:
        for j in i:
            bigramfeatures.append(j.tolist())
bigramfeatures=torch.tensor(bigramfeatures)
trainsent=df1['word_sentiment'].apply(lambda i:[int(j) for j in ast.literal_eval(i)])

In [231]:
#Test Text Bigrams
class testtextbigrams:
    def __init__(self,df):
        self.df=df
        self.text=self.preprocess(df['raw'])
    def __len__(self):
        return len(self.text)
    def __getitem__(self,ind):
        text=self.text[ind]
        return text
    def preprocess(self,text):
        tt=[]
        for i in text:
            tt.append(cleantext.clean(str(i),clean_all=True,stemming=True,stopwords=True,lowercase=True,numbers=True,punct=True))
        bigram=[]
        for i in tt:
            j=word_tokenize(i)
            bigrams=list(ngrams(j,2))
            bigram.append(bigrams)
        index={}
        for i in bigram:
            for j in i:
                if j not in index:
                    index[j]=len(index)
        embedded=[]
        for i in bigram:
            emb=torch.tensor([index[j] for j in i])
            embedded.append(emb)
        padded=pad_sequence(embedded,batch_first=True,padding_value=0)
        return padded
df2=df[(df['split']=='test')]
df2=df2.reset_index()
dataset=textbigrams(df2)
testdataloader=DataLoader(dataset,batch_size=32,shuffle=False)
testbigramfeatures=[]
with torch.no_grad():
    for i in testdataloader:
        for j in i:
            testbigramfeatures.append(j.tolist())
testbigramfeatures=torch.tensor(testbigramfeatures)
testsent=df2['word_sentiment'].apply(lambda i:[int(j) for j in ast.literal_eval(i)])

In [232]:
#Val Text Bigrams
class valbigrams:
    def __init__(self,df):
        self.df=df
        self.text=self.preprocess(df['raw'])
    def __len__(self):
        return len(self.text)
    def __getitem__(self,ind):
        text=self.text[ind]
        return text
    def preprocess(self,text):
        tt=[]
        for i in text:
            tt.append(cleantext.clean(str(i),clean_all=True,stemming=True,stopwords=True,lowercase=True,numbers=True,punct=True))
        bigram=[]
        for i in tt:
            j=word_tokenize(i)
            bigrams=list(ngrams(j,2))
            bigram.append(bigrams)
        index={}
        for i in bigram:
            for j in i:
                if j not in index:
                    index[j]=len(index)
        embedded=[]
        for i in bigram:
            emb=torch.tensor([index[j] for j in i])
            embedded.append(emb)
        padded=pad_sequence(embedded,batch_first=True,padding_value=0)
        return padded
df3=df[(df['split']=='val')]
df3=df3.reset_index()
dataset=valbigrams(df3)
valdataloader=DataLoader(dataset,batch_size=32,shuffle=False)
valbigramfeatures=[]
with torch.no_grad():
    for i in valdataloader:
        for j in i:
            valbigramfeatures.append(j.tolist())
valbigramfeatures=torch.tensor(valbigramfeatures)
valsent=df3['word_sentiment'].apply(lambda i:[int(j) for j in ast.literal_eval(i)])

In [233]:
#Test Sentiments
testsentimentbigram=[]
for i in testsent:
    bigrams=list(ngrams(i,2))
    testsentimentbigram.append(bigrams)
testsentiment=[]
for k in testsentimentbigram:
    for i in k:
        m=[]
        for j in range(0,len(i)-1):
            if i[j]==0 and i[j+1]==0:
                m=[0]
            elif i[j]==0 and i[j+1]==1:
                m=[0]
            elif i[j]==1 and i[j+1]==0:
                m=[0]
            elif i[j]==1 and i[j+1]==1:
                m=[1]
    testsentiment.append(m)
sent_test=[]
for i in testsentiment:
    t=torch.tensor(i)
    sent_test.append(t)

In [234]:
#Val Sentiments
valsentimentbigram=[]
for i in valsent:
    bigrams=list(ngrams(i,2))
    valsentimentbigram.append(bigrams)
valsentiment=[]
for k in valsentimentbigram:
    for i in k:
        m=[]
        for j in range(0,len(i)-1):
            if i[j]==0 and i[j+1]==0:
                m=[0]
            elif i[j]==0 and i[j+1]==1:
                m=[0]
            elif i[j]==1 and i[j+1]==0:
                m=[0]
            elif i[j]==1 and i[j+1]==1:
                m=[1]
    valsentiment.append(m)
sent_val=[]
for i in valsentiment:
    t=torch.tensor(i)
    sent_val.append(t)

In [235]:
#Train Sentiments
sentimentbigram=[]
for i in trainsent:
    bigrams=list(ngrams(i,2))
    sentimentbigram.append(bigrams)
sentiment=[]
for k in sentimentbigram:
    for i in k:
        m=[]
        for j in range(0,len(i)-1):
            if i[j]==0 and i[j+1]==0:
                m=[0]
            elif i[j]==0 and i[j+1]==1:
                m=[0]
            elif i[j]==1 and i[j+1]==0:
                m=[0]
            elif i[j]==1 and i[j+1]==1:
                m=[1]
    sentiment.append(m)
sent_train=[]
for i in sentiment:
    t=torch.tensor(i)
    sent_train.append(t)

In [236]:
#DataLoader and Dataset
valbigramfeatures=F.pad(valbigramfeatures,(0,45-valbigramfeatures.size(1)))
bigramfeatures=F.pad(bigramfeatures,(0,45-bigramfeatures.size(1)))
class Text_sent:
    def __init__(self,text,sent):
        self.text=text
        self.sent=sent
    def __len__(self):
        return len(self.text)
    def __getitem__(self,index):
        text=self.text[index]
        sent=self.sent[index]
        return text,sent
dataset=Text_sent(bigramfeatures,sent_train)
CombineDataloader=DataLoader(dataset,batch_size=43,shuffle=False)
testdataset=Text_sent(testbigramfeatures,sent_test)
testCombineDataloader=DataLoader(testdataset,batch_size=43,shuffle=False)
valdataset=Text_sent(valbigramfeatures,sent_val)
valCombineDataloader=DataLoader(valdataset,batch_size=43,shuffle=False)

In [237]:
#Model
class bigramtextmodel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1=nn.Linear(45,320)
        self.act_fn=nn.Sigmoid()
        self.linear2=nn.Linear(320,256)
        self.linear3=nn.Linear(256,128)
        self.linear4=nn.Linear(128,64)
        self.linear5=nn.Linear(64,32)
        self.linear6=nn.Linear(32,16)
        self.linear7=nn.Linear(16,8)
        self.linear8=nn.Linear(8,2)
    def forward(self,x):
        x=x.to(torch.float32)
        x=self.linear1(x)
        x=self.act_fn(x)
        x=self.linear2(x)
        x=self.act_fn(x)
        x=self.linear3(x)
        x=self.act_fn(x)
        x=self.linear4(x)
        x=self.act_fn(x)
        x=self.linear5(x)
        x=self.act_fn(x)
        x=self.linear6(x)
        x=self.act_fn(x)
        x=self.linear7(x)
        x=self.act_fn(x)
        x=self.linear8(x)
        return x
tokenmodel=bigramtextmodel()

In [238]:
#Training
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(tokenmodel.parameters(),lr=0.001)
criterion2=nn.CrossEntropyLoss()
optimizer2=torch.optim.Adam(tokenmodel.parameters(),lr=0.01)
for epoch in range(10):
    tokenmodel.train()
    trainloss=0.0
    for texts,labels in CombineDataloader:
        labels=labels.view(-1)
        outputs=tokenmodel(texts)
        loss=criterion(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        trainloss=trainloss+loss.item()
    tokenmodel.eval()
    vloss=0.0
    with torch.no_grad():
        for texts,labels in valCombineDataloader:
            labels=labels.view(-1)
            optimizer2.zero_grad()
            outputs=tokenmodel(texts)
            loss=criterion2(outputs,labels)
            vloss=vloss+loss.item()
    print('Training Loss :',trainloss)
    print('Validation Loss: ',vloss)

Training Loss : 325.37596678733826
Validation Loss:  60.28992587327957
Training Loss : 288.70098972320557
Validation Loss:  56.48875394463539
Training Loss : 279.93030121922493
Validation Loss:  55.41835060715675
Training Loss : 277.70346570014954
Validation Loss:  55.07523164153099
Training Loss : 277.1168564558029
Validation Loss:  54.95061707496643
Training Loss : 276.96174651384354
Validation Loss:  54.899831384420395
Training Loss : 276.9222120940685
Validation Loss:  54.87705147266388
Training Loss : 276.91324415802956
Validation Loss:  54.86609101295471
Training Loss : 276.91191217303276
Validation Loss:  54.86056697368622
Training Loss : 276.9121915102005
Validation Loss:  54.857705384492874


In [239]:
#Evaluate
#Saving the Model
torch.save(tokenmodel.state_dict(),"trained_model3.pth")
#Evaluate
tokenmodel=bigramtextmodel()
tokenmodel.load_state_dict(torch.load("trained_model3.pth"))
tokenmodel.eval()
pred=[]
label=[]
with torch.no_grad():
    for texts,labels in testCombineDataloader:
        labels=labels.view(-1)
        outputs=tokenmodel(texts)
        for i,j in zip(torch.argmax(outputs,axis=1),labels):
            pred.append(i.tolist())
            label.append(j.tolist())
accuracy=metrics.accuracy_score(label,pred)
precision=metrics.precision_score(label,pred,zero_division=1)
recall=metrics.recall_score(label,pred)
f1score=metrics.f1_score(label,pred,average='macro')
print('Accuracy :',accuracy*100)
print('Recall :',recall)
print('Precision :',precision)
print('F1 Score :',f1score)

Accuracy : 69.947209653092
Recall : 0.0
Precision : 1.0
F1 Score : 0.41158198358109604


In [240]:
#Loss
torch.save(model.state_dict(),"trained_model2.pth")
tokenmodel=bigramtextmodel()
criteria=nn.CrossEntropyLoss()
model.load_state_dict(torch.load("trained_model2.pth"))
tokenmodel.eval()
loss=0.0
with torch.no_grad():
    for texts,labels in testCombineDataloader:
        labels=labels.view(-1)
        outputs=tokenmodel(texts)
        losss=criteria(outputs,labels)
        loss=loss+losss.item()
print("Test Loss: ",loss)

Test Loss:  230.1139524281025
